In [2]:
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

from shapely.geometry import box
from shapely.ops import clip_by_rect
from skfem import Basis, ElementTriP0
from skfem.io.meshio import from_meshio

from femwell.maxwell.waveguide import compute_modes
from femwell.mesh import mesh_from_OrderedDict

In [3]:
wavelength = 1.55  # um 
thickness = 0.4    # um (400 nm) 
num_modes = 20
n_core = 2.0       # SiN refractive index 
n_clad = 1.444     # SiO2 refractive index (Standard assumption)

In [ ]:
widths = np.linspace(0.5, 2.0, 25)

te0_neffs = []
te1_neffs = []

print(f"Starting sweep for SiN Waveguide (Thickness={thickness}um, n={n_core})...")

# --- 2. Run the Sweep ---
for width in widths:
    core = box(-width/2, -thickness/2, width/2, thickness/2)

    # Define refractive indices for the mesh
    polygons = OrderedDict(
        core = core,
        box=clip_by_rect(core.buffer(1.0, resolution=4), -np.inf, -np.inf, np.inf, 0),
        clad=clip_by_rect(core.buffer(1.0, resolution=4), -np.inf, 0, np.inf, np.inf),
    )

    resolutions = {"core": {"resolution": 0.1, "distance": 1}}

    # Generate the Finite Element Mesh
    mesh = from_meshio(mesh_from_OrderedDict(polygons, resolutions, default_resolution_max=0.6))

    basis0 = Basis(mesh, ElementTriP0())
    epsilon = basis0.zeros(dtype=complex)
    for subdomain, n in {"core": 1.9963, "box": 1.444, "clad": 1}.items():
        epsilon[basis0.get_dofs(elements=subdomain)] = n**2

    modes = compute_modes(basis0, epsilon, wavelength=wavelength, num_modes=num_modes)

    # Let's extract all real n_effs to analyze
    n_effs_real = [np.real(mode.n_eff) for mode in modes]
    te0_neffs.append(n_effs_real[0]) # Fundamental mode

    # Check for higher order modes
    # A mode is guided if n_eff > n_clad
    guided_modes = [n for n in n_effs_real if n > n_clad]
    print(f"Guided modes for width = {width}: {guided_modes}")

Starting sweep for SiN Waveguide (Thickness=0.4um, n=2.0)...
Guided modes for width = 0.5: []
Guided modes for width = 0.5526315789473684: []
Guided modes for width = 0.6052631578947368: [np.float64(1.4638549950298312)]
Guided modes for width = 0.6578947368421053: [np.float64(1.4841545685064252)]
Guided modes for width = 0.7105263157894737: [np.float64(1.5042381223954582)]
Guided modes for width = 0.763157894736842: [np.float64(1.5220236541134347)]
Guided modes for width = 0.8157894736842105: [np.float64(1.539227097462348), np.float64(1.4478840574263212)]
Guided modes for width = 0.868421052631579: [np.float64(1.5534610771583197), np.float64(1.454582907376194)]
Guided modes for width = 0.9210526315789473: [np.float64(1.5672680671996346), np.float64(1.4610229087169138)]
Guided modes for width = 0.9736842105263157: [np.float64(1.5779434147112477), np.float64(1.4671461219230075)]
Guided modes for width = 1.026315789473684: [np.float64(1.5891563106849478), np.float64(1.4723425507819996)]
G